In [1]:

import seaborn as sb
import matplotlib as mp
import os
import glob
import pandas as pd
import tqdm
from DataLoader_pytorch import window_torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch

/Users/hyunsucksong/opt/anaconda3/envs/songforgpu/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_size, shift = 1, 1
train = []
test = []
val = []

using_columns = ['Time[s]', 'TE1', 'E.RPM',
       'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar','E.Id','RPM', 'P_UD']


#x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar']
x_columns = [ 'TE1' ]
print(len(x_columns))


y_columns = ['P_UD']



1


In [3]:
train_datapath = 'csv_final/train'
test_datapath = "csv_final/test"
val_datapath = "csv_final/val"
train_data = os.listdir(train_datapath)
test_data = os.listdir(test_datapath)
val_data = os.listdir(val_datapath)


In [4]:
for csv_file in tqdm.tqdm(train_data) : 
    
    df_cut = pd.read_csv('csv_final/train/' + csv_file)[using_columns]
    train.append(df_cut)

for csv_file in tqdm.tqdm(test_data) : 
    
    df_cut = pd.read_csv('csv_final/test/' + csv_file)[using_columns]
    test.append(df_cut)
    
for csv_file in tqdm.tqdm(val_data) : 
    
    df_cut = pd.read_csv('csv_final/val/' + csv_file)[using_columns]
    val.append(df_cut)

100%|███████████████████████████████████████████| 44/44 [00:02<00:00, 21.50it/s]


In [22]:
print(train)

[      Time[s]     TE1   E.RPM  E.Vlt  E.Iq  E.Tmp  E.Pwr   E.Tar  E.Id  RPM  \
0       0.000  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
1       0.001  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
2       0.002  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
3       0.003  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
4       0.004  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
...       ...     ...     ...    ...   ...    ...    ...     ...   ...  ...   
9213    9.213  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9214    9.214  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9215    9.215  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9216    9.216  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9217    9.217  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   

       P_UD  
0     1.543  
1     1.545  
2     1.

In [18]:
train_X, train_Y = window_torch(train, x_size, shift, x_columns, y_columns)
train_dataset = torch.utils.data.TensorDataset(train_X, train_Y)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=False) 

test_X, test_Y = window_torch(test, x_size, shift, x_columns, y_columns)
test_dataset = torch.utils.data.TensorDataset(test_X, test_Y)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False) 

val_X, val_Y = window_torch(val, x_size, shift, x_columns, y_columns)
val_dataset = torch.utils.data.TensorDataset(val_X, val_Y)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)

In [25]:
print(train_dataset)

In [19]:
learning_rate = 0.001

num_epochs = 1000

num_x_columes = len(x_columns)

#model = nn.Linear(num_x_columes,1)
model = nn.Linear(num_x_columes,1)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [29]:
def Lrmodel(num_epochs, model, optimizer, train_dl, loss_fn):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            #print("x: ", xb)
            #print("y: ", yb)
            pred = model(xb)
            loss = loss_fn(pred, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print('Epochs : [{}]/[{}],  MSE Loss : {}'.format(epoch + 1, num_epochs, loss.item()))

In [ ]:
Lrmodel(num_epochs, model, optimizer, train_loader, loss_fn)

Epochs : [1]/[1000],  MSE Loss : 0.0006192580331116915
Epochs : [2]/[1000],  MSE Loss : 0.0006192636792548001
Epochs : [3]/[1000],  MSE Loss : 0.0006192636792548001
Epochs : [4]/[1000],  MSE Loss : 0.0006192636792548001
Epochs : [5]/[1000],  MSE Loss : 0.0006192750297486782
Epochs : [6]/[1000],  MSE Loss : 0.0006192750297486782
Epochs : [7]/[1000],  MSE Loss : 0.0006192807340994477


### 